In [2]:
import py_entitymatching as em
import pandas as pd
import os, sys

In [3]:
A = em.read_csv_metadata('amazon_books_utf8.csv')
B = em.read_csv_metadata('walmart_books_utf8.csv')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
A['id'] = range(0, len(A))
em.set_key(A, 'id')
B['id'] = range(0, len(B))
em.set_key(B, 'id')

True

In [5]:
sep1 = '–'
sep2 = ':'
sep3 = ')'
sep4 = '('
A['Name'] = A['Name'].apply(lambda x: x.split(sep1, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.split(sep2, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.split(sep3, 1)[0])
A['Name'] = A['Name'].apply(lambda x: x.replace(sep4, ''))
A['Publisher'] = A['Publisher'].apply(lambda x: x.split(sep4, 1)[0])
B['Name'] = B['Name'].apply(lambda x: x.split(sep2, 1)[0])

In [30]:
def name_similarity(x, y):
    if len(x['Name'].split()) == 1 and len(y['Name'].split()) == 1:
        if x['Name'].lower().strip() == y['Name'].lower().strip():
            return False
    return True

In [ ]:
# Instantiate blackbox blocker
bb = em.BlackBoxBlocker()
# Set the black box function
bb.set_black_box_function(name_similarity)
C1 = bb.block_tables(A, B, l_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'], r_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'] )

In [166]:
ob = em.OverlapBlocker()
C2 = ob.block_tables(A, B, 'Author', 'Author',word_level=True, overlap_size=2, l_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'], r_output_attrs=['Name', 'Sale Price', 'Category', 'Author', 'ISBN10', 'Pages', 'Publisher', 'Language', 'Dimensions', 'Weight', 'Rating'] )
#C1 = ob.block_tables(A, B, 'Author', 'Author',word_level=True, overlap_size=2, l_output_attrs=['Name', 'Author', 'ISBN10'], r_output_attrs=['Name','Author', 'ISBN10'] )
C3 = ob.block_candset(C2, 'Name', 'Name',overlap_size=2,word_level=True, rem_stop_words=True)
C4 = em.combine_blocker_outputs_via_union([C1,C3])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [167]:
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
block_f

Column Weight does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column Weight


,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,Name_Name_jac_qgm_3_qgm_3,Name,Name,qgm_3,qgm_3,jaccard,<function Name_Name_jac_qgm_3_qgm_3 at 0x000002C651B89D08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,Name_Name_cos_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,cosine,<function Name_Name_cos_dlm_dc0_dlm_dc0 at 0x000002C6525466A8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,Name_Name_jac_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,jaccard,<function Name_Name_jac_dlm_dc0_dlm_dc0 at 0x000002C651D73400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,Name_Name_mel,Name,Name,None,None,monge_elkan,<function Name_Name_mel at 0x000002C651D73840>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Name_Name_lev_dist,Name,Name,None,None,lev_dist,<function Name_Name_lev_dist at 0x000002C651D73E18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Name_Name_lev_sim,Name,Name,None,None,lev_sim,<function Name_Name_lev_sim at 0x000002C651D73EA0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Name_Name_nmw,Name,Name,None,None,needleman_wunsch,<function Name_Name_nmw at 0x000002C651D73730>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Name_Name_sw,Name,Name,None,None,smith_waterman,<function Name_Name_sw at 0x000002C651D73620>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Sale_Price_Sale_Price_exm,Sale Price,Sale Price,None,None,exact_match,<function Sale_Price_Sale_Price_exm at 0x000002C651D730D0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Sale_Price_Sale_Price_anm,Sale Price,Sale Price,None,None,abs_norm,<function Sale_Price_Sale_Price_anm at 0x000002C651D73048>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [198]:
rb = em.RuleBasedBlocker()
rb.add_rule(['Pages_Pages_anm(ltuple, rtuple) < 0.8'], block_f)
C5 = rb.block_candset(C4, show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [202]:
C5.to_csv('blocked_pairs.csv', index = False, sep = ',')

In [199]:
E = ob.block_candset(C5, 'ISBN10', 'ISBN10',overlap_size=1,word_level=True, rem_stop_words=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [200]:
E

,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Sale Price,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating
0,0,4,39,The Kremlin Conspiracy Hardcover,18.29,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Joel C. Rosenberg,1496406176,NaN,"Tyndale House Publishers, Inc.",...,21.61,Books > Summer Reading Books > Beach Reads,Joel C. Rosenberg,1496406176,480.0,"Tyndale House Publishers, Inc.",English,9.00 x 6.00 x 1.25 Inches,NaN,NaN
2,2,8,2334,Origin,16.77,Books > Literature & Fiction > Action & Adventure,Dan Brown,385514239,NaN,Doubleday; 1st Edition edition,...,16.50,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...","Brown, Dan",385514239,461.0,Random House Inc,English,9.50 x 6.50 x 1.75 Inches,NaN,4.0000
3,3,9,306,We Were the Lucky Ones,8.97,Books > Literature & Fiction > Genre Fiction,Georgia Hunter,399563091,416.0,Penguin Books; Reprint edition,...,8.97,Books > Literature & Fiction > Fiction > Historical,Georgia Hunter,399563091,416.0,Penguin BooksPenguin Publishing Group,English,8.25 x 5.50 x 1.10 Inches (US),NaN,NaN
6,6,11,1301,"Sing, Unburied, Sing",14.79,Books > Literature & Fiction > Genre Fiction,Jesmyn Ward,1501126067,NaN,Scribner; First Edition/First Printing edition,...,21.24,Books > Literature & Fiction > Fiction > Literary,Jesmyn Ward,1501126067,304.0,Scribner,English,8.38 x 139.70 x 1.20 Inches (US),NaN,NaN
30,30,20,1641,Artemis,16.63,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Andy Weir,553448129,NaN,Crown; F First Edition edition,...,16.63,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > Action & ...,"Weir, Andy",553448129,305.0,Random House Inc,English,9.50 x 6.25 x 1.25 Inches,NaN,NaN
35,35,30,571,The Rooster Bar Hardcover,15.28,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",John Grisham,385541171,NaN,Doubleday; 1st Edition edition,...,15.28,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...",John Grisham,385541171,368.0,Bantam Dell,English,9.25 x 6.12 x 1.06 Inches,NaN,NaN
37,37,34,112,The Wife Between Us,16.19,Books > Literature & Fiction > Women's Fiction,Greer Hendricks Sarah Pekkanen,1250130921,NaN,St. Martin's Press,...,21.28,Books > Women's History Month Books > Women's History Month Fiction,Sarah Pekkanen Greer Hendricks,1250130921,352.0,St. Martin's Press,English,9.25 x 155.57 x 33.27 Inches (US),NaN,5.0000
42,42,37,2342,The Light We Lost Paperback,10.07,Books > Literature & Fiction > Women's Fiction,Jill Santopolo,735212767,352.0,G.P. Putnam's Sons; Reprint edition,...,5.36,Books > Literature & Fiction > Fiction > Contemporary Women,Jill Santopolo,735212767,352.0,Penguin Publishing GroupG.P. Putnam's Sons,English,NaN,NaN,NaN
44,44,38,2066,The Handmaid's Tale Paperback,9.59,Books > Literature & Fiction > Genre Fiction,Margaret Atwood,038549081X,311.0,Anchor; 1st Anchor Books edition,...,9.59,Books > Walmart Best Sellers > Best seller ranking: > # > 5401 > > in > Books > # > 447 > > in...,"ATWOOD, MARGARET",038549081X,336.0,Random House Inc,English,8.00 x 5.25 x 0.75 Inches,NaN,4.5000
47,47,40,737,Still Me,17.70,Books > Literature & Fiction > Genre Fiction,Jojo Moyes,399562451,NaN,Pamela Dorman Books,...,16.20,Books > Literature & Fiction > Fiction > Contemporary Women,Jojo Moyes,399562451,400.0,Penguin Publishing GroupPamela Dorman Books,English,9.00 x 6.00 x 1.00 Inches (US),NaN,NaN


In [201]:
E
E.to_csv('blocked_pairs_ISBN_match.csv', index = False, sep = ',')

In [23]:
 dbg = em.debug_blocker(C5, A, B)

In [24]:
dbg[['ltable_Name','ltable_ISBN10','ltable_Author','rtable_Name','rtable_ISBN10','rtable_Author']]

,ltable_Name,ltable_ISBN10,ltable_Author,rtable_Name,rtable_ISBN10,rtable_Author
0,The Broken Road,1501111647,Richard Paul Evans,The Mistletoe Promise,1442372680,Richard Paul Evans
1,Persepolis Rising The Expanse,316332836,James S. A. Corey,Cibola Burn,6334685,James S. A. Corey
2,Persepolis Rising The Expanse,316332836,James S. A. Corey,Caliban's War,6129069,James S. A. Corey
3,Persepolis Rising The Expanse,316332836,James S. A. Corey,Abaddon's Gate,6129074,James S. A. Corey
4,Persepolis Rising The Expanse,316332836,James S. A. Corey,Nemesis Games,6334711,James S. A. Corey
5,Persepolis Rising The Expanse,316332836,James S. A. Corey,Babylon's Ashes,316217646,James S. A. Corey
6,Persepolis Rising The Expanse,316332836,James S. A. Corey,Babylon's Ashes,031633474X,James S. A. Corey
7,The Kremlin Conspiracy Hardcover,1496406176,Joel C. Rosenberg,Without Warning,1496423291,Joel C. Rosenberg
8,The Kremlin Conspiracy Hardcover,1496406176,Joel C. Rosenberg,Dead Heat,4311624,Joel C. Rosenberg
9,The Kremlin Conspiracy Hardcover,1496406176,Joel C. Rosenberg,The J. B. Collins Collection,1496427718,Joel C. Rosenberg


In [79]:
S = em.sample_table(C5, 400)
G = em.label_table(S, label_column_name='gold_labels')

Column name (gold_labels) is not present in dataframe


In [80]:
G

,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating,gold_labels
56,56,1767,3,"Saga, Vol. 2 Paperback",9.37,Books > Comics & Graphic Novels > Graphic Novels,Brian K. Vaughan,1607066920,144.0,Image Comics,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > General,"Staples, Fiona Vaughan, Brian K.",2150783,504.0,Diamond Comic Distributors,English,8.00 x 10.00 x 1.00 Inches,NaN,5.0,0
58,58,3116,3,Saga Volume 6 Paperback,10.19,Books > Science Fiction & Fantasy > Science Fiction,Brian K Vaughan,163215711X,152.0,Image Comics,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > General,"Staples, Fiona Vaughan, Brian K.",2150783,504.0,Diamond Comic Distributors,English,8.00 x 10.00 x 1.00 Inches,NaN,5.0,0
356,356,762,14,Along Came A Spider Mass Market Paperback,6.48,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",James Patterson,446364193,528.0,Grand Central Publishing,...,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...",James Patterson,6364196,528.0,Grand Central Pub Mass Market,English,6.75 x 4.00 x 1.00 Inches,NaN,NaN,1
497,497,257,17,"The Wheel of Time, Boxed Set II, Books 4-6",16.32,Books > Science Fiction & Fantasy > Fantasy,Robert Jordan,812540115,NaN,Tor Books; 1 edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic,Robert Jordan,2550293,704.0,Tom Doherty AssociatesTor Fantasy,English,6.76 x 104.65 x 1.56 Inches (US),NaN,5.0,0
600,600,3085,20,The Old Man and The Sea Paperback,9.94,Books > Literature & Fiction > Genre Fiction,Ernest Hemingway,684801221,128.0,Scribner; Reissue edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > General,"Cline, Ernest",99586746,NaN,Cornerstone,NaN,NaN,NaN,NaN,0
606,606,1052,21,Etched in Bone A Novel of the Others,8.99,Books > Literature & Fiction > Genre Fiction,Anne Bishop,451474503,528.0,Roc; Reissue edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > General,"McCaffrey, Anne",5467219,480.0,Random House,English,7.00 x 4.25 x 1.25 Inches,NaN,NaN,0
610,610,2083,21,The Dragonriders of Pern,15.10,Books > Science Fiction & Fantasy > Fantasy,Anne McCaffrey,345340248,864.0,Del Rey; Reprint edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Science Fiction > General,"McCaffrey, Anne",5467219,480.0,Random House,English,7.00 x 4.25 x 1.25 Inches,NaN,NaN,0
643,643,849,24,"A is for Alibi Kinsey Millhone Alphabet Mysteries, No. 1",14.57,"Books > Mystery, Thriller & Suspense > Mystery",Sue Grafton,312353812,320.0,St. Martin's Griffin; Reprint edition,...,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Mystery & Detective >...",Sue Grafton,2353814,320.0,St. Martin's PressSt. Martin's Griffin,English,8.16 x 142.49 x 22.35 Inches (US),NaN,NaN,1
663,663,2845,26,"Living Dead in Dallas Sookie Stackhouse/True Blood, Book 2",5.99,Books > Literature & Fiction > Genre Fiction,Charlaine Harris,441009239,291.0,Ace Books,...,Books > Literature & Fiction > Fiction > African American,"Harris, K. D. Harris, Kd",982391307,200.0,Cartel Publications,English,5.50 x 0.60 x 8.40 Inches,NaN,NaN,0
824,824,59,32,The Couple Next Door,12.80,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Shari Lapena,735221103,336.0,Penguin Books; Reprint edition,...,Books > Literature & Fiction > Fiction > Psychological,"Lapena, Shari",143284010X,435.0,Christian Large Print,NaN,5.50 x 8.50 x 1.00 Inches,NaN,NaN,1


In [81]:
G.to_csv('labeled_data.csv', index = False, sep = ',')

In [82]:
train_test = em.split_train_test(G, train_proportion=0.5,random_state=0)
I = train_test['train']
J = train_test['test']

I.to_csv('train.csv')
J.to_csv('test.csv')

In [146]:
F = em.get_features_for_matching(A, B)

Column Weight does not seem to qualify as any atomic type. It may contain all NaNs. Please update the values of column Weight


The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
1,Sale Price,Sale Price,numeric,numeric,Exact Match; Absolute Norm
2,Category,Category,medium string (5 words to 10 words),short string (1 word),Not Applicable: Types do not match
3,Author,Author,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,ISBN10,ISBN10,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
5,Pages,Pages,numeric,numeric,Exact Match; Absolute Norm
6,Publisher,Publisher,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Language,Language,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
8,Dimensions,Dimensions,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
9,Weight,Weight,short string (1 word to 5 words),un-determined type,Not Applicable: Types do not match


Do you want to proceed? (y/n):y


In [147]:
F

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,Name_Name_jac_qgm_3_qgm_3,Name,Name,qgm_3,qgm_3,jaccard,<function Name_Name_jac_qgm_3_qgm_3 at 0x000001CAE59B97B8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,Name_Name_cos_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,cosine,<function Name_Name_cos_dlm_dc0_dlm_dc0 at 0x000001CAE7399378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,Name_Name_jac_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,jaccard,<function Name_Name_jac_dlm_dc0_dlm_dc0 at 0x000001CAE59AE9D8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,Name_Name_mel,Name,Name,None,None,monge_elkan,<function Name_Name_mel at 0x000001CAE59AEC80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Name_Name_lev_dist,Name,Name,None,None,lev_dist,<function Name_Name_lev_dist at 0x000001CAE59AE2F0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Name_Name_lev_sim,Name,Name,None,None,lev_sim,<function Name_Name_lev_sim at 0x000001CAE59AEB70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Name_Name_nmw,Name,Name,None,None,needleman_wunsch,<function Name_Name_nmw at 0x000001CAE59AE400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Name_Name_sw,Name,Name,None,None,smith_waterman,<function Name_Name_sw at 0x000001CAE59AEA60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,Sale_Price_Sale_Price_exm,Sale Price,Sale Price,None,None,exact_match,<function Sale_Price_Sale_Price_exm at 0x000001CAE59AED90>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,Sale_Price_Sale_Price_anm,Sale Price,Sale Price,None,None,abs_norm,<function Sale_Price_Sale_Price_anm at 0x000001CAE59AEBF8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [148]:
F = F.drop(F.index[[8,9,10,11,44,45,46,47,48,49,50,51,52,53,54,55,56]])
F

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,Name_Name_jac_qgm_3_qgm_3,Name,Name,qgm_3,qgm_3,jaccard,<function Name_Name_jac_qgm_3_qgm_3 at 0x000001CAE59B97B8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,Name_Name_cos_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,cosine,<function Name_Name_cos_dlm_dc0_dlm_dc0 at 0x000001CAE7399378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,Name_Name_jac_dlm_dc0_dlm_dc0,Name,Name,dlm_dc0,dlm_dc0,jaccard,<function Name_Name_jac_dlm_dc0_dlm_dc0 at 0x000001CAE59AE9D8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,Name_Name_mel,Name,Name,None,None,monge_elkan,<function Name_Name_mel at 0x000001CAE59AEC80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,Name_Name_lev_dist,Name,Name,None,None,lev_dist,<function Name_Name_lev_dist at 0x000001CAE59AE2F0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,Name_Name_lev_sim,Name,Name,None,None,lev_sim,<function Name_Name_lev_sim at 0x000001CAE59AEB70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,Name_Name_nmw,Name,Name,None,None,needleman_wunsch,<function Name_Name_nmw at 0x000001CAE59AE400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,Name_Name_sw,Name,Name,None,None,smith_waterman,<function Name_Name_sw at 0x000001CAE59AEA60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
12,Author_Author_jac_qgm_3_qgm_3,Author,Author,qgm_3,qgm_3,jaccard,<function Author_Author_jac_qgm_3_qgm_3 at 0x000001CAE59AE488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
13,Author_Author_cos_dlm_dc0_dlm_dc0,Author,Author,dlm_dc0,dlm_dc0,cosine,<function Author_Author_cos_dlm_dc0_dlm_dc0 at 0x000001CAE59AEF28>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [149]:
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after='gold_labels',
                            show_progress=False) 

# Display first few rows
H.head()

,_id,ltable_id,rtable_id,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Name_Name_nmw,...,Publisher_Publisher_lev_sim,Publisher_Publisher_nmw,Publisher_Publisher_sw,Language_Language_lev_dist,Language_Language_lev_sim,Language_Language_jar,Language_Language_jwn,Language_Language_exm,Language_Language_jac_qgm_3_qgm_3,gold_labels
21981,21981,2728,812,0.156863,0.471405,0.285714,0.671103,28.0,0.300000,-12.0,...,0.354839,5.0,9.0,0.0,1.0,1.0,1.0,1.0,1.0,0
41794,41794,70,1503,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,20.0,...,0.297872,-17.0,13.0,0.0,1.0,1.0,1.0,1.0,1.0,1
39363,39363,2260,1414,0.086207,0.218218,0.111111,0.612884,37.0,0.212766,-21.0,...,0.160000,-4.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0
47784,47784,645,1724,0.000000,0.000000,0.000000,0.430556,10.0,0.166667,-2.0,...,0.131579,-21.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0
48268,48268,132,1750,0.000000,0.000000,0.000000,0.421652,16.0,0.111111,-3.0,...,0.153846,-5.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0


In [150]:
any(pd.notnull(H))

True

In [151]:
H.fillna(value=0, inplace=True)

In [152]:
dt = em.DTMatcher(name='DecisionTree', random_state=0,max_depth=5)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [153]:
result_precision = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='precision', random_state=0)
result_recall = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='recall', random_state=0)
result_f1 = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric_to_select_matcher='f1', random_state=0)
result_precision['cv_stats']
result_recall['cv_stats']
result_f1['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.796667,0.852857,0.808485
1,RF,0.905556,0.855000,0.850420
2,SVM,0.000000,0.000000,0.000000
3,LinReg,0.916667,0.776429,0.813187
4,LogReg,0.784762,0.704286,0.715455
5,NaiveBayes,0.570256,0.870000,0.670327


In [154]:
# Running The matcher

In [155]:
L = em.extract_feature_vecs(J, feature_table=F,
                            attrs_after='gold_labels', show_progress=False)

In [157]:
any(pd.notnull(L))
L.fillna(value=0, inplace=True)

In [159]:
L

,_id,ltable_id,rtable_id,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,Name_Name_mel,Name_Name_lev_dist,Name_Name_lev_sim,Name_Name_nmw,...,Publisher_Publisher_nmw,Publisher_Publisher_sw,Language_Language_lev_dist,Language_Language_lev_sim,Language_Language_jar,Language_Language_jwn,Language_Language_exm,Language_Language_jac_qgm_3_qgm_3,gold_labels,predicted_labels
21267,21267,1062,790,0.160714,0.471405,0.222222,0.832727,46.0,0.163636,-37.0,...,-6.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
56363,56363,573,2028,0.132075,0.000000,0.000000,0.649934,18.0,0.437500,7.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
63076,63076,1308,2261,0.137931,0.353553,0.200000,0.569795,11.0,0.352941,1.0,...,-7.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
34656,34656,2524,1216,0.055556,0.188982,0.100000,0.467186,32.0,0.179487,-18.0,...,-6.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
43945,43945,2954,1588,0.105263,0.182574,0.100000,0.594907,25.0,0.218750,2.0,...,2.0,7.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
8741,8741,2220,301,0.184211,0.447214,0.285714,0.787176,16.0,0.360000,1.0,...,-11.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
25919,25919,3094,955,0.000000,0.000000,0.000000,0.461480,20.0,0.130435,-1.0,...,-7.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
45705,45705,2187,1655,0.000000,0.000000,0.000000,0.503053,22.0,0.214286,-9.0,...,0.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
22346,22346,2631,828,0.122807,0.471405,0.285714,0.515188,33.0,0.195122,-15.0,...,5.0,9.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0
38213,38213,1901,1372,0.295455,0.632456,0.400000,0.865000,27.0,0.325000,-14.0,...,-17.0,6.0,0.0,1.0,1.0,1.0,1.0,1.0,1,0


In [160]:
J

,_id,ltable_id,rtable_id,ltable_Name,ltable_Sale Price,ltable_Category,ltable_Author,ltable_ISBN10,ltable_Pages,ltable_Publisher,...,rtable_Category,rtable_Author,rtable_ISBN10,rtable_Pages,rtable_Publisher,rtable_Language,rtable_Dimensions,rtable_Weight,rtable_Rating,gold_labels
21267,21267,1062,790,Lee Child Jack Reacher Books 1-3 Mass Market Paperback,21.93,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Lee Child,515154113,NaN,Berkley; Box edition,...,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...","Child, Lee",NaN,484.0,Random House,English,5.50 x 0.50 x 7.44 Inches,NaN,5.0000,0
56363,56363,573,2028,The Boy on the Bridge Paperback,13.89,Books > Science Fiction & Fantasy > Science Fiction,M. R. Carey,316300349,432.0,Orbit; Reprint edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > General,"Tolkien, J. R. R.",8205531,NaN,HarperCollins Publishers,NaN,NaN,NaN,NaN,0
63076,63076,1308,2261,Of Blood and Bone,23.79,"Books > Mystery, Thriller & Suspense > Thrillers & Suspense",Nora Roberts,1250122996,NaN,St. Martin's Press,...,Books > Literature & Fiction > Fiction > Romance > Contemporary,Nora Roberts,5152914,336.0,BerkleyPenguin Publishing Group,English,NaN,NaN,NaN,0
34656,34656,2524,1216,The Book of Us,11.39,"Books > Reference > Writing, Research & Publishing Guides",David Marshall Kate Marshall,078686477X,NaN,Hachette Books,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > General,"E, David",132944745X,34.0,Lulu.com,English,6.00 x 0.08 x 9.00 Inches,NaN,NaN,0
43945,43945,2954,1588,Dance Upon the Air Three Sisters,10.95,Books > Literature & Fiction > Women's Fiction,Nora Roberts,042527814X,352.0,Berkley; Reprint edition,...,Books > Literature & Fiction > Fiction > Romance > Suspense > Best seller ranking: > # > 5470 > ...,"Roberts, Nora",5146015,1.0,Berkley Pub Group,English,7.00 x 4.25 x 2.75 Inches,NaN,5.0000,0
8741,8741,2220,301,The City of Brass,15.39,Books > Literature & Fiction > Genre Fiction,S. A Chakraborty,62678108,544.0,Harper Voyager,...,Books > Literature & Fiction > Fiction > Westerns,Joseph A. West Ralph Compton,1235169,304.0,BerkleyPenguin Publishing Group,English,NaN,NaN,NaN,0
25919,25919,3094,955,The World of Ice & Fire,35.99,Books > Literature & Fiction > Genre Fiction,George R. R. Martin Elio Garcia Linda Antonsson,553805444,NaN,Bantam; Later Printing edition,...,Books > Literature & Fiction > Fiction > Science Fiction & Fantasy > Fantasy > Epic > Best selle...,"SALVATORE, R. A.",696517X,352.0,Wizards of the Coast,English,6.75 x 3.50 x 1.25 Inches,NaN,NaN,0
45705,45705,2187,1655,The Way Of The Shadow Wolves,14.11,"Books > Mystery, Thriller & Suspense > Mystery",Tom Morrissey,999497502,240.0,5th Palace Publishing,...,"Books > Literature & Fiction > Fiction > Mysteries, Thrillers & Suspense > Thrillers & Suspense ...","Clancy, Tom",514758X,990.0,Berkley Pub Group,English,6.75 x 4.25 x 1.50 Inches,NaN,NaN,0
22346,22346,2631,828,Vengeance in Death Mass Market Paperback,7.99,"Books > Mystery, Thriller & Suspense > Mystery",J. D. Robb,425160394,384.0,Berkley; Reissue edition,...,Books > Literature & Fiction > Fiction > Romance > Suspense,J. D. Robb,5279000,400.0,BerkleyPenguin Publishing Group,English,6.75 x 4.19 x 1.06 Inches (US),NaN,NaN,0
38213,38213,1901,1372,The Alchemist,13.33,Books > Comics & Graphic Novels > Graphic Novels,Paulo Coelho,62024329,208.0,HarperOne; Later Printing Used edition,...,Books > Literature & Fiction > Fiction > Literary > Best seller ranking: > # > 413 > > in > Boo...,"COELHO, PAULO",2315009,208.0,Harpercollins,English,5.50 x 8.00 x 0.75 Inches,NaN,5.0000,1


In [161]:
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels','predicted_labels'], 
                         append=True,target_attr='predicted_labels')
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 100.0% (25/25)
Recall : 75.76% (25/33)
F1 : 86.21%
False positives : 0 (out of 25 positive predictions)
False negatives : 8 (out of 175 negative predictions)


In [162]:
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels','predicted_labels'], 
                         append=True,target_attr='predicted_labels')
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 88.46% (23/26)
Recall : 69.7% (23/33)
F1 : 77.97%
False positives : 3 (out of 26 positive predictions)
False negatives : 10 (out of 174 negative predictions)
